# NER with Bert

Source: https://medium.com/@andrewmarmon/fine-tuned-named-entity-recognition-with-hugging-face-bert-d51d4cb3d7b5

In [27]:
import os
import re
import itertools
import pandas as pd
import numpy as np
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch

In [2]:
label_list = ['O','B-MISC','I-MISC','B-PER','I-PER','B-ORG','I-ORG','B-LOC','I-LOC']
label_encoding_dict = {'I-PRG': 2,'I-I-MISC': 2, 'I-OR': 6,  'I-': 0, 'VMISC': 0, # line covers ann=tation erros in train file 
                       'O': 0, 'B-MISC': 1, 'I-MISC': 2, 'B-PER': 3, 'I-PER': 4, 'B-ORG': 5, 'I-ORG': 6, 'B-LOC': 7, 'I-LOC': 8,}

task = "ner" 
model_checkpoint = "distilbert-base-uncased"
batch_size = 16
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [3]:
def get_all_tokens_and_ner_tags(directory):
    return pd.concat([get_tokens_and_ner_tags(os.path.join(directory, filename)) for filename in os.listdir(directory)]).reset_index().drop('index', axis=1)
    
def get_tokens_and_ner_tags(filename):
    with open(filename, 'r', encoding="utf8") as f:
        lines = f.readlines()
        split_list = [list(y) for x, y in itertools.groupby(lines, lambda z: z == '\n') if not x]
        tokens = [[x.split('\t')[0] for x in y] for y in split_list]
        entities = [[x.split('\t')[1][:-1] for x in y] for y in split_list] 
    return pd.DataFrame({'tokens': tokens, 'ner_tags': entities})
  
def get_un_token_dataset(train_directory, test_directory):
    train_df = get_all_tokens_and_ner_tags(train_directory)
    test_df = get_all_tokens_and_ner_tags(test_directory)
    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)

    return (train_dataset, test_dataset)

In [4]:
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [5]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] 
                         for (p, l) in zip(prediction, label) 
                         if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], 
            "recall": results["overall_recall"], "f1": results["overall_f1"], 
            "accuracy": results["overall_accuracy"]}

## Read annotated data

In [6]:
train_dataset, test_dataset = get_un_token_dataset('../../data/UN-named-entity-recognition/tagged-training/', 
                                                   '../../data/UN-named-entity-recognition/tagged-test/')

In [7]:
train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized_datasets = test_dataset.map(tokenize_and_align_labels, batched=True)

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 19.07ba/s]


## Train model

In [8]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=1e-5,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

In [9]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=test_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()
trainer.save_model('un-ner.model')

The following columns in the training set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, ner_tags. If tokens, ner_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
/home/erikt/projects/rel20/venv3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3657
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 687


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.052608,0.789836,0.850255,0.818933,0.982015
2,No log,0.049980,0.830493,0.858600,0.844313,0.984916
3,0.063400,0.057054,0.828901,0.866945,0.847496,0.985039


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, ner_tags. If tokens, ner_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2074
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, ner_tags. If tokens, ner_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2074
  Batch size = 16
Saving model checkpoint to test-ner/checkpoint-500
Configuration saved in test-ner/checkpoint-500/config.json
Model weights saved in test-ner/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-500/tokenizer_config.json
Special tokens file saved in t

Saving model checkpoint to un-ner.model
Configuration saved in un-ner.model/config.json
Model weights saved in un-ner.model/pytorch_model.bin
tokenizer config file saved in un-ner.model/tokenizer_config.json
Special tokens file saved in un-ner.model/special_tokens_map.json


## Test model

In [10]:
tokenizer = AutoTokenizer.from_pretrained('./un-ner.model/')

paragraph = '''
    Before proceeding further, I should like to inform members that action on draft 
    resolution iv, entitled situation of human rights of Rohingya Muslims and other 
    minorities in Myanmar is postponed to a later date to allow time for the review 
    of its programme budget implications by the fifth committee. The assembly will 
    take action on draft resolution iv as soon as the report of the fifth committee 
    on the programme budget implications is available. I now give the floor to 
    delegations wishing to deliver explanations of vote or position before voting or 
    adoption.'''
tokens = tokenizer(paragraph)
torch.tensor(tokens['input_ids']).unsqueeze(0).size()

Didn't find file ./un-ner.model/added_tokens.json. We won't load it.
loading file ./un-ner.model/vocab.txt
loading file ./un-ner.model/tokenizer.json
loading file None
loading file ./un-ner.model/special_tokens_map.json
loading file ./un-ner.model/tokenizer_config.json


torch.Size([1, 103])

In [11]:
model = AutoModelForTokenClassification.from_pretrained('./un-ner.model/', num_labels=len(label_list))
predictions = model.forward(input_ids=torch.tensor(tokens['input_ids']).unsqueeze(0), attention_mask=torch.tensor(tokens['attention_mask']).unsqueeze(0))
predictions = torch.argmax(predictions.logits.squeeze(), axis=1)
predictions = [label_list[i] for i in predictions]

loading configuration file ./un-ner.model/config.json
Model config DistilBertConfig {
  "_name_or_path": "./un-ner.model/",
  "activation": "gelu",
  "architectures": [
    "DistilBertForTokenClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "

In [12]:
words = tokenizer.batch_decode(tokens['input_ids'])
pd.DataFrame({'ner': predictions, 'words': words}).to_csv('un_ner.csv')

# bert large NER

Source: https://huggingface.co/dslim/bert-large-NER

In [13]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-large-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

loading configuration file https://huggingface.co/dslim/bert-large-NER/resolve/main/config.json from cache at /home/erikt/.cache/huggingface/transformers/5a0d81781c7e4fc475ede69166dd515e7c5b54df30bae5185d675057f9352ff8.70a92b0531aa1dfc5ed255c8492db74ef24c17365b0fe86f1fc3ef4e3f028cd2
Model config BertConfig {
  "_name_or_path": "dslim/bert-large-NER",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "eos_token_ids": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "O",
    "1": "B-MISC",
    "2": "I-MISC",
    "3": "B-PER",
    "4": "I-PER",
    "5": "B-ORG",
    "6": "I-ORG",
    "7": "B-LOC",
    "8": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "B-LOC": 7,
    "B-MISC": 1,
    "B-ORG": 5,
    "B-PER": 3,
    "I-LOC": 8,
    "I-MISC": 2,
    "I-ORG": 6,
    "I-PER": 4,
    "

[{'entity': 'B-PER', 'score': 0.9971501, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.9986046, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


In [14]:
example = "SOCCER - GERMAN FIRST DIVISION SUMMARIES. BONN 1996-12-06 Summaries of matches played in the German first division on Friday : Bochum 2 (Stickroth 30th pen, Wosz 89th) Bayer Leverkusen 2 (Kirsten 18th, Ramelow 56th). Halftime 1-1. Attendance : 24,602 Werder Bremen 1 (Bode 85th) 1860 Munich 1 (Bormirow 8th). Halftime 0-1. Attendance 33,000 Karlsruhe 3 (Reich 29th, Carl 44th, Dundee 69th) Freiburg 0. Halftime 2-0. Attendance 33,000 Schalke 2 (Mulder 2nd and 27th) Hansa Rostock 0. Halftime 2-0. Attendance 29,300"

In [44]:
ner_results = nlp(example)
print(ner_results)

[{'entity': 'B-MISC', 'score': 0.99924064, 'index': 6, 'word': 'GE', 'start': 9, 'end': 11}, {'entity': 'I-MISC', 'score': 0.8793608, 'index': 7, 'word': '##R', 'start': 11, 'end': 12}, {'entity': 'I-MISC', 'score': 0.884922, 'index': 8, 'word': '##MA', 'start': 12, 'end': 14}, {'entity': 'I-MISC', 'score': 0.9773442, 'index': 9, 'word': '##N', 'start': 14, 'end': 15}, {'entity': 'B-MISC', 'score': 0.9992536, 'index': 40, 'word': 'German', 'start': 93, 'end': 99}, {'entity': 'B-ORG', 'score': 0.9987205, 'index': 46, 'word': 'Bo', 'start': 127, 'end': 129}, {'entity': 'I-ORG', 'score': 0.99158394, 'index': 47, 'word': '##chu', 'start': 129, 'end': 132}, {'entity': 'I-ORG', 'score': 0.9947837, 'index': 48, 'word': '##m', 'start': 132, 'end': 133}, {'entity': 'B-PER', 'score': 0.998579, 'index': 51, 'word': 'Stick', 'start': 137, 'end': 142}, {'entity': 'B-PER', 'score': 0.44963565, 'index': 52, 'word': '##roth', 'start': 142, 'end': 146}, {'entity': 'B-PER', 'score': 0.9985172, 'index': 

In [65]:
def combine_entities(ner_results):
    ner_results_out = []
    i = 0
    while i < len(ner_results)-1:
        last_end = ner_results[i]["end"]
        ner_results_out.append(dict(ner_results[i]))
        j = 1
        while i + j < len(ner_results) and (ner_results[i+j]["start"] == last_end or
                                            (ner_results[i+j]["start"] == last_end + 1 and 
                                             re.search("^I", ner_results[i+j]["entity"]) and
                                             re.sub("^..", "", ner_results[i+j]["entity"]) == re.sub("^..", "", ner_results[i]["entity"]))):
            if ner_results[i+j]["start"] == last_end:
                ner_results_out[-1]["word"] += re.sub("^##", "", ner_results[i+j]["word"])
            else:
                ner_results_out[-1]["word"] += " " + ner_results[i+j]["word"]
            ner_results_out[-1]["end"] = ner_results[i+j]["end"]
            last_end = ner_results[i+j]["end"]
            j += 1
        i += j
    return ner_results_out

In [66]:
combine_entities(ner_results)

[{'entity': 'B-MISC',
  'score': 0.99924064,
  'index': 6,
  'word': 'GERMAN',
  'start': 9,
  'end': 15},
 {'entity': 'B-MISC',
  'score': 0.9992536,
  'index': 40,
  'word': 'German',
  'start': 93,
  'end': 99},
 {'entity': 'B-ORG',
  'score': 0.9987205,
  'index': 46,
  'word': 'Bochum',
  'start': 127,
  'end': 133},
 {'entity': 'B-PER',
  'score': 0.998579,
  'index': 51,
  'word': 'Stickroth',
  'start': 137,
  'end': 146},
 {'entity': 'B-PER',
  'score': 0.9985172,
  'index': 56,
  'word': 'Wosz',
  'start': 157,
  'end': 161},
 {'entity': 'B-ORG',
  'score': 0.99935025,
  'index': 62,
  'word': 'Bayer Leverkusen',
  'start': 168,
  'end': 184},
 {'entity': 'B-PER',
  'score': 0.99836916,
  'index': 70,
  'word': 'Kirsten',
  'start': 188,
  'end': 195},
 {'entity': 'B-PER',
  'score': 0.9985753,
  'index': 74,
  'word': 'Ramelow',
  'start': 202,
  'end': 209},
 {'entity': 'B-ORG',
  'score': 0.99929786,
  'index': 93,
  'word': 'Werder Bremen',
  'start': 251,
  'end': 264},
